In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

# Load the dataset
data = pd.read_csv("Per (1).csv")

# Split the data into input features (X) and output variable (y)
X = data.drop(columns=['Gender', 'Age', 'Personality', 'Type', 'Mindset', 'Occasion','Perfume'])
y = data['Perfume']

# One-hot encode the categorical features
X = pd.get_dummies(X, prefix_sep='_')

# Encode the output variable
le = LabelEncoder()
y = le.fit_transform(y)

# Train the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=100)
model.fit(X, y)

# Save the model to a pickle file
with open('perfume_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Load the model from the pickle file
with open('perfume_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Define a function to process user inputs and make predictions
def process_user_inputs(inputs):
    # Convert the user inputs to a DataFrame
    user_df = pd.DataFrame(inputs, index=[0])
    
    # One-hot encode the categorical features
    user_df = pd.get_dummies(user_df, prefix_sep='_')
    
    # Reorder the columns to match the training data
    missing_cols = set(X.columns) - set(user_df.columns)
    for col in missing_cols:
        user_df[col] = 0
    user_df = user_df[X.columns]
    
    # Make the prediction
    user_pred = loaded_model.predict(user_df)[0]
    
    # Decode the prediction
    user_perfume = le.inverse_transform([user_pred])[0]
    
    # Return the recommended perfume
    return user_perfume


In [2]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pickle
from http.server import BaseHTTPRequestHandler, HTTPServer
import json

# Load the trained model
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

# Encode the output variable
le = LabelEncoder()

class RequestHandler(BaseHTTPRequestHandler):
    # Handle POST requests
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)
        data = json.loads(post_data)

        # Get user inputs from the form
        gender = data['gender']
        age = data['age']
        frequency = data['frequency']
        personality = data['personality']
        type = data['type']
        mindset = data['mindset']
        occasion = data['occasion']

        # Convert the user inputs to a DataFrame
        user_df = pd.DataFrame({
            'Gender': [gender],
            'Age': [age],
            'Frequency': [frequency],
            'Personality': [personality],
            'Type': [type],
            'Mindset': [mindset],
            'Occasion': [occasion]
        })

        # One-hot encode the categorical features
        user_df = pd.get_dummies(user_df, prefix_sep='_')

        # Make sure user inputs match the model's features
        user_df = user_df.reindex(columns=X.columns, fill_value=0)

        # Make the prediction
        user_pred = model.predict(user_df)[0]

        # Decode the prediction
        user_perfume = le.inverse_transform([user_pred])[0]

        # Prepare the response data
        response_data = {'perfume': user_perfume}

        # Set the response status code
        self.send_response(200)

        # Set the response headers
        self.send_header('Content-type', 'application/json')
        self.end_headers()

        # Send the response data
        self.wfile.write(json.dumps(response_data).encode())

# Load the dataset
data = pd.read_csv("Per (1).csv")

# Split the data into input features (X) and output variable (y)
X = data.drop(columns=['Gender', 'Age', 'Personality', 'Type', 'Mindset', 'Occasion','Perfume'])
y = data['Perfume']

# One-hot encode the categorical features
X = pd.get_dummies(X, prefix_sep='_')

# Encode the output variable
le.fit(y)

# Create an instance of the HTTP server
server = HTTPServer(('localhost', 8000), RequestHandler)
print('Server started on port 8000')

# Start the server
server.serve_forever()


Server started on port 8000


KeyboardInterrupt: 

In [7]:
import http.server
import json

class PerfumeRequestHandler(http.server.BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        body = self.rfile.read(content_length)
        user_inputs = json.loads(body)

        # Extract user inputs from 'user_inputs' dictionary
        gender = user_inputs['gender']
        # Extract values of other input fields

        # Call the recommend_perfume function
        recommended_perfume = recommend_perfume(gender, age, frequency, personality, type, mindset, occasion)

        # Prepare the JSON response
        response = {'perfume': recommended_perfume}
        response_json = json.dumps(response).encode('utf-8')

        # Set response headers
        self.send_response(200)
        self.send_header('Content-Type', 'application/json')
        self.send_header('Content-Length', len(response_json))
        self.end_headers()

        # Send the response
        self.wfile.write(response_json)

# Run the server
server_address = ('', 8000)
httpd = http.server.HTTPServer(server_address, PerfumeRequestHandler)
httpd.serve_forever()


KeyboardInterrupt: 

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load the dataset
data = pd.read_csv("Per (1).csv")

# Split the data into input features (X) and output variable (y)
X = data.drop(columns=['Gender', 'Age', 'Personality', 'Type', 'Mindset', 'Occasion', 'Perfume'])
y = data['Perfume']

# One-hot encode the categorical features
X = pd.get_dummies(X, prefix_sep='_')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

# Encode the output variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Train the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=100)
model.fit(X_train, y_train)

# Save the trained model to a file
joblib.dump(model, 'model.pkl')


['model.pkl']

In [ ]:
!python server.ipynb



In [ ]:
import http.server
import socketserver

handler = http.server.SimpleHTTPRequestHandler
httpd = socketserver.TCPServer(("", 8000), handler)

print("Server started on port 8000")
httpd.serve_forever()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import json

# Load the dataset
data = pd.read_csv("Per (1).csv")

# Split the data into input features (X) and output variable (y)
#X = data.drop('Perfume', axis=1)
X = data.drop(columns=['Gender', 'Age', 'Personality', 'Type', 'Mindset', 'Occasion','Perfume'])

y = data['Perfume']
# One-hot encode the categorical features
X = pd.get_dummies(X, prefix_sep='_')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

# Encode the output variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Train the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=100)
model.fit(X_train, y_train)

# Define a function to take user inputs and return a predicted perfume
def recommend_perfume(user_input):
    user_df = pd.DataFrame(user_input)
    # One-hot encode the categorical features
    user_df = pd.get_dummies(user_df, prefix_sep='_')

    # Reorder the columns to match the training data
    missing_cols = set(X.columns) - set(user_df.columns)
    for col in missing_cols:
        user_df[col] = 0
    user_df = user_df[X.columns]

    # Make the prediction
    user_pred = model.predict(user_df)[0]

    # Decode the prediction
    user_perfume = le.inverse_transform([user_pred])[0]

    # Return the recommended perfume
    return user_perfume

# Import the necessary libraries for creating a simple HTTP server
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse as urlparse

class RequestHandler(BaseHTTPRequestHandler):
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        if self.path == '/':
            with open('index.html', 'rb') as file:
                self._set_response()
                self.wfile.write(file.read())
        else:
            self.send_error(404, 'File Not Found: %s' % self.path)

    def do_POST(self):
        if self.path == '/recommend':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            user_input = json.loads(post_data)

            perfume = recommend_perfume(user_input)

            self._set_response()
            self.wfile.write(json.dumps({'perfume': perfume}).encode('utf-8'))
        else:
            self.send_error(404, 'File Not Found: %s' % self.path)

# Set up the server
server_address = ('', 8000)
httpd = HTTPServer(server_address, RequestHandler)
print('Starting server on port 8000...')

# Start the server
httpd.serve_forever()


Starting server on port 8000...
